# Import Depedency

In [1]:
import pandas as pd

import numpy as np
import plotly.graph_objects as go

import plotly.express as px
from plotly.subplots import make_subplots

import utils


# Read CSV Data

In [2]:
data = pd.read_csv("../data/skygeni_sales_data.csv")
data.head()

,deal_id,created_date,closed_date,sales_rep_id,industry,region,product_type,lead_source,deal_stage,deal_amount,sales_cycle_days,outcome
0,D00001,2023-11-24,2023-12-15,rep_22,SaaS,North America,Enterprise,Referral,Qualified,4253,21,Won
1,D00002,2023-01-17,2023-01-27,rep_7,SaaS,India,Core,Referral,Closed,3905,10,Won
2,D00003,2023-10-29,2023-12-10,rep_5,HealthTech,APAC,Core,Inbound,Proposal,10615,42,Lost
3,D00004,2023-07-14,2023-08-02,rep_18,FinTech,India,Core,Partner,Negotiation,4817,19,Won
4,D00005,2024-02-29,2024-05-26,rep_2,HealthTech,APAC,Core,Outbound,Qualified,45203,87,Lost


# Data Types Check

In [3]:
data.dtypes

deal_id             object
created_date        object
closed_date         object
sales_rep_id        object
industry            object
region              object
product_type        object
lead_source         object
deal_stage          object
deal_amount          int64
sales_cycle_days     int64
outcome             object
dtype: object

## Update Date Date type

In [4]:
data = utils.update_date_dtype(data)

## Categorical Fields To Labels Encoding

In [5]:
data = utils.categorical_encoding(data)

In [6]:
data.head()

,deal_id,created_date,closed_date,sales_rep_id,industry,region,product_type,lead_source,deal_stage,deal_amount,sales_cycle_days,outcome
0,D00001,2023-11-24,2023-12-15,1,1,1,1,1,1,4253,21,1
1,D00002,2023-01-17,2023-01-27,2,1,2,2,1,2,3905,10,1
2,D00003,2023-10-29,2023-12-10,3,2,3,2,2,3,10615,42,0
3,D00004,2023-07-14,2023-08-02,4,3,2,2,3,4,4817,19,1
4,D00005,2024-02-29,2024-05-26,5,2,3,2,4,1,45203,87,0


In [7]:
data.columns

Index(['deal_id', 'created_date', 'closed_date', 'sales_rep_id', 'industry',
       'region', 'product_type', 'lead_source', 'deal_stage', 'deal_amount',
       'sales_cycle_days', 'outcome'],
      dtype='object')

In [8]:

col_grid = [
    ['sales_rep_id', 'industry'],
    ['region', 'product_type'], 
    ['lead_source', 'deal_stage'], 
    ['deal_amount', 'sales_cycle_days']
]

utils.plot_dist_grid(data, col_grid=col_grid, bins=25)

In [9]:
# =========================
# SALES FUNNEL / WIN-RATE EDA KIT
# =========================

data = pd.read_csv("../data/skygeni_sales_data.csv")
data = utils.update_date_dtype(data)
data["outcome"].replace({"Lost":0, "Won":1}, inplace=True)

# -------------------------
# 0) PREPROCESS
# -------------------------
def preprocess_sales_df(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()

    # parse timestamps
    d["created_date"] = pd.to_datetime(d["created_date"], errors="coerce")
    d["closed_date"]  = pd.to_datetime(d["closed_date"],  errors="coerce")

    # ensure outcome int
    d["outcome"] = pd.to_numeric(d["outcome"], errors="coerce").astype("Int64")

    # deal_amount numeric
    d["deal_amount"] = pd.to_numeric(d["deal_amount"], errors="coerce")

    # sales_cycle_days numeric
    if "sales_cycle_days" in d.columns:
        d["sales_cycle_days"] = pd.to_numeric(d["sales_cycle_days"], errors="coerce")

    # helpful month keys (Period)
    d["created_month"] = d["created_date"].dt.to_period("M")
    d["closed_month"]  = d["closed_date"].dt.to_period("M")

    # flags
    d["is_won"]  = (d["outcome"] == 1).astype(int)
    d["is_lost"] = (d["outcome"] == 0).astype(int)

    return d


# -------------------------
# 1) BASIC HEALTH CHECKS
# -------------------------
def eda_overview(d: pd.DataFrame):
    print("Shape:", d.shape)
    print("\nMissing %:")
    miss = (d.isna().mean().sort_values(ascending=False) * 100).round(2)
    print(miss[miss > 0])

    print("\nOutcome distribution:")
    print(d["outcome"].value_counts(dropna=False))

    # date coverage
    print("\nCreated date range:", d["created_date"].min(), "->", d["created_date"].max())
    print("Closed  date range:", d["closed_date"].min(),  "->", d["closed_date"].max())

    # duplicates
    if "deal_id" in d.columns:
        dup = d["deal_id"].duplicated().sum()
        print("\nDuplicate deal_id:", dup)


# -------------------------
# 2) MONTHLY + CUMULATIVE KPI TRENDS
# -------------------------
def monthly_kpis(
    d: pd.DataFrame,
    time_basis="closed_month",   # "closed_month" or "created_month"
    amount_col="deal_amount"
) -> pd.DataFrame:
    if time_basis not in ["closed_month", "created_month"]:
        raise ValueError("time_basis must be 'closed_month' or 'created_month'")

    # select month
    m = d.dropna(subset=[time_basis]).copy()
    m["month"] = m[time_basis].dt.to_timestamp()

    grp = m.groupby("month", as_index=False).agg(
        deals=("deal_id","nunique") if "deal_id" in m.columns else ("outcome","size"),
        won=("is_won","sum"),
        lost=("is_lost","sum"),
        win_rate=("is_won","mean"),
        total_amount=(amount_col,"sum"),
        avg_amount=(amount_col,"mean"),
        median_amount=(amount_col,"median"),
        avg_cycle_days=("sales_cycle_days","mean") if "sales_cycle_days" in m.columns else (amount_col, "size"),
    )

    # cumulative
    grp = grp.sort_values("month")
    grp["cum_deals"] = grp["deals"].cumsum()
    grp["cum_won"]   = grp["won"].cumsum()
    grp["cum_lost"]  = grp["lost"].cumsum()
    grp["cum_win_rate"] = grp["cum_won"] / grp["cum_deals"]
    grp["cum_total_amount"] = grp["total_amount"].cumsum()
    grp["cum_won_amount"] = (
        m.assign(month=m["month"])
         .groupby("month")["deal_amount"]
         .sum()
         .reindex(grp["month"])
         .fillna(0)
         .cumsum()
         .values
    )

    return grp


def plot_monthly_kpis(kpi: pd.DataFrame, title_prefix="Monthly KPIs"):
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            "Deals & Won/Lost (monthly)",
            "Win rate (monthly)",
            "Deal amount (monthly)",
            "Cumulative win rate"
        ],
        vertical_spacing=0.14, horizontal_spacing=0.10
    )

    # (1) deals + won/lost
    fig.add_trace(go.Bar(x=kpi["month"], y=kpi["deals"], name="Deals"), row=1, col=1)
    fig.add_trace(go.Bar(x=kpi["month"], y=kpi["won"],   name="Won", opacity=0.7), row=1, col=1)
    fig.add_trace(go.Bar(x=kpi["month"], y=kpi["lost"],  name="Lost", opacity=0.7), row=1, col=1)

    # (2) win rate monthly
    fig.add_trace(go.Scatter(x=kpi["month"], y=kpi["win_rate"], mode="lines+markers", name="Win rate"), row=1, col=2)

    # (3) amount monthly
    fig.add_trace(go.Scatter(x=kpi["month"], y=kpi["total_amount"], mode="lines+markers", name="Total amount"), row=2, col=1)
    fig.add_trace(go.Scatter(x=kpi["month"], y=kpi["avg_amount"], mode="lines+markers", name="Avg amount"), row=2, col=1)

    # (4) cumulative win rate
    fig.add_trace(go.Scatter(x=kpi["month"], y=kpi["cum_win_rate"], mode="lines+markers", name="Cum win rate"), row=2, col=2)

    fig.update_layout(
        template="plotly_white",
        barmode="overlay",
        height=800,
        width=1200,
        title=f"{title_prefix}",
        legend_title_text="Metric"
    )
    fig.update_yaxes(tickformat=".0%", row=1, col=2)
    fig.update_yaxes(tickformat=".0%", row=2, col=2)
    fig.show()


# -------------------------
# 3) WIN-RATE BY CATEGORY (OVERALL + MONTHLY CHANGE)
# -------------------------
def top_categories(d: pd.DataFrame, col: str, top_n=10):
    return d[col].value_counts(dropna=False).head(top_n).index.tolist()

def category_winrate_overall(d: pd.DataFrame, col: str, top_n=15, min_n=30):
    x = d.dropna(subset=[col, "outcome"]).copy()
    # group
    g = x.groupby(col).agg(
        n=("outcome","size"),
        win_rate=("is_won","mean"),
        avg_amount=("deal_amount","mean")
    ).reset_index()

    g = g[g["n"] >= min_n].sort_values("win_rate", ascending=False)
    if top_n:
        g = pd.concat([g.head(top_n), g.tail(top_n)]).drop_duplicates()

    fig = px.bar(
        g.sort_values("win_rate", ascending=True),
        x="win_rate", y=col, orientation="h",
        hover_data=["n","avg_amount"],
        title=f"Win rate by {col} (min_n={min_n})"
    )
    fig.update_layout(template="plotly_white", height=600)
    fig.update_xaxes(tickformat=".0%")
    fig.show()
    return g

def category_winrate_monthly(d: pd.DataFrame, col: str, time_basis="closed_month", top_n=10, min_n_per_month=10):
    m = d.dropna(subset=[col, time_basis, "outcome"]).copy()
    m["month"] = m[time_basis].dt.to_timestamp()
    # restrict to top categories overall for readability
    cats = top_categories(m, col, top_n=top_n)
    m = m[m[col].isin(cats)]

    g = m.groupby(["month", col], as_index=False).agg(
        n=("outcome","size"),
        win_rate=("is_won","mean"),
        avg_amount=("deal_amount","mean")
    )
    g = g[g["n"] >= min_n_per_month]

    fig = px.line(
        g, x="month", y="win_rate", color=col,
        markers=True,
        hover_data=["n","avg_amount"],
        title=f"Monthly win rate trend by {col} (top {top_n}, min_n_per_month={min_n_per_month})"
    )
    fig.update_layout(template="plotly_white", height=550, width=1200)
    fig.update_yaxes(tickformat=".0%")
    fig.show()
    return g

def category_winrate_heatmap(d: pd.DataFrame, col: str, time_basis="closed_month", top_n=12, min_n_per_cell=8):
    m = d.dropna(subset=[col, time_basis, "outcome"]).copy()
    m["month"] = m[time_basis].dt.to_timestamp()
    cats = top_categories(m, col, top_n=top_n)
    m = m[m[col].isin(cats)]

    g = m.groupby(["month", col]).agg(
        n=("outcome","size"),
        win_rate=("is_won","mean")
    ).reset_index()

    g = g[g["n"] >= min_n_per_cell]

    pivot = g.pivot(index=col, columns="month", values="win_rate").sort_index()
    fig = px.imshow(
        pivot,
        aspect="auto",
        title=f"Heatmap: Win rate by {col} across months (top {top_n})"
    )
    fig.update_layout(template="plotly_white", height=550, width=1200)
    fig.show()
    return g


# -------------------------
# 4) CONTINUOUS EFFECTS: deal_amount & cycle_days vs win
# -------------------------
import pandas as pd
import plotly.express as px

def continuous_effects(d: pd.DataFrame, time_basis="closed_month"):
    x = d.dropna(subset=["deal_amount", "outcome"]).copy()

    # Amount distribution by outcome
    fig1 = px.histogram(
        x, x="deal_amount", color="outcome",
        nbins=40, barmode="overlay",
        title="Deal amount distribution by outcome"
    )
    fig1.update_layout(template="plotly_white")
    fig1.show()

    # Win rate by amount deciles (✅ convert Interval -> str)
    x["amount_decile"] = pd.qcut(x["deal_amount"], q=10, duplicates="drop")
    x["amount_decile_str"] = x["amount_decile"].astype(str)

    g = x.groupby("amount_decile_str", as_index=False).agg(
        n=("outcome","size"),
        win_rate=("is_won","mean"),
        avg_amount=("deal_amount","mean")
    )

    # keep order by avg_amount (so bins appear increasing)
    g = g.sort_values("avg_amount")

    fig2 = px.bar(
        g, x="amount_decile_str", y="win_rate",
        hover_data=["n","avg_amount"],
        title="Win rate by deal_amount decile"
    )
    fig2.update_layout(template="plotly_white")
    fig2.update_yaxes(tickformat=".0%")
    fig2.show()

    # Cycle days impact (if present)
    if "sales_cycle_days" in x.columns:
        y = d.dropna(subset=["sales_cycle_days", "outcome"]).copy()
        y["cycle_bin"] = pd.qcut(y["sales_cycle_days"], q=10, duplicates="drop")
        y["cycle_bin_str"] = y["cycle_bin"].astype(str)

        gy = y.groupby("cycle_bin_str", as_index=False).agg(
            n=("outcome","size"),
            win_rate=("is_won","mean"),
            avg_cycle=("sales_cycle_days","mean")
        ).sort_values("avg_cycle")

        fig3 = px.bar(
            gy, x="cycle_bin_str", y="win_rate",
            hover_data=["n","avg_cycle"],
            title="Win rate by sales_cycle_days decile"
        )
        fig3.update_layout(template="plotly_white")
        fig3.update_yaxes(tickformat=".0%")
        fig3.show()



# -------------------------
# 5) MULTIVARIATE: WIN RATE BY (CATEGORY x CATEGORY)
# -------------------------
def winrate_pivot(d: pd.DataFrame, row_cat: str, col_cat: str, min_n=30):
    x = d.dropna(subset=[row_cat, col_cat, "outcome"]).copy()
    g = x.groupby([row_cat, col_cat]).agg(
        n=("outcome","size"),
        win_rate=("is_won","mean")
    ).reset_index()

    # mask small cells
    g.loc[g["n"] < min_n, "win_rate"] = np.nan

    pivot = g.pivot(index=row_cat, columns=col_cat, values="win_rate")
    fig = px.imshow(pivot, aspect="auto", title=f"Win rate heatmap: {row_cat} x {col_cat} (min_n={min_n})")
    fig.update_layout(template="plotly_white", height=650, width=1200)
    fig.show()

    return g


# -------------------------
# 6) WHAT CHANGED? (MONTH-OVER-MONTH DELTAS)
# -------------------------
def mom_changes(kpi: pd.DataFrame):
    z = kpi.copy().sort_values("month")
    for c in ["deals", "won", "lost", "win_rate", "total_amount", "avg_amount", "median_amount"]:
        if c in z.columns:
            z[f"{c}_mom_abs"] = z[c].diff()
            # safe pct change
            z[f"{c}_mom_pct"] = z[c].pct_change().replace([np.inf, -np.inf], np.nan)
    return z

def plot_mom_changes(z: pd.DataFrame):
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=["Deals MoM %", "Win rate MoM (abs)", "Total amount MoM %", "Avg amount MoM %"],
        vertical_spacing=0.14
    )

    if "deals_mom_pct" in z:
        fig.add_trace(go.Bar(x=z["month"], y=z["deals_mom_pct"], name="Deals MoM %"), row=1, col=1)
    if "win_rate_mom_abs" in z:
        fig.add_trace(go.Bar(x=z["month"], y=z["win_rate_mom_abs"], name="Win rate MoM abs"), row=1, col=2)
    if "total_amount_mom_pct" in z:
        fig.add_trace(go.Bar(x=z["month"], y=z["total_amount_mom_pct"], name="Total amount MoM %"), row=2, col=1)
    if "avg_amount_mom_pct" in z:
        fig.add_trace(go.Bar(x=z["month"], y=z["avg_amount_mom_pct"], name="Avg amount MoM %"), row=2, col=2)

    fig.update_layout(template="plotly_white", height=800, width=1200, title="Month-over-Month Changes")
    fig.show()


# -------------------------
# 7) OPTIONAL: SIMPLE DRIVER MODEL (LOGISTIC REGRESSION)
#    - tells which features associate with wins
# -------------------------
def logistic_driver_model(d: pd.DataFrame, time_basis="closed_month", drop_cols=("deal_id",)):
    """
    Quick baseline model:
      - One-hot encodes categoricals
      - Uses deal_amount + sales_cycle_days + categoricals
      - Returns top coefficients (directional only; not causal)
    """
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score

    x = d.dropna(subset=["outcome"]).copy()

    # basic feature set
    feature_cols = [c for c in x.columns if c not in set(["outcome"]) and c not in set(drop_cols)]
    # remove raw dates but keep month if helpful
    for c in ["created_date", "closed_date"]:
        if c in feature_cols:
            feature_cols.remove(c)

    # add month as categorical (optional, helps detect time drift)
    if time_basis in x.columns:
        x["month_feat"] = x[time_basis].astype(str)
        feature_cols.append("month_feat")

    y = x["outcome"].astype(int)
    X = x[feature_cols]

    # detect categorical vs numeric
    cat_cols = [c for c in X.columns if X[c].dtype == "object" or str(X[c].dtype).startswith("category")]
    # also treat these as categorical
    for c in ["industry","region","product_type","lead_source","deal_stage","sales_rep_id","month_feat"]:
        if c in X.columns and c not in cat_cols:
            cat_cols.append(c)

    num_cols = [c for c in X.columns if c not in cat_cols]
    # keep only numeric-like in num
    num_cols = [c for c in num_cols if pd.api.types.is_numeric_dtype(X[c])]

    pre = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
            ("num", "passthrough", num_cols),
        ],
        remainder="drop",
    )

    clf = LogisticRegression(max_iter=2000, n_jobs=None)
    pipe = Pipeline([("pre", pre), ("clf", clf)])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    pipe.fit(X_train, y_train)
    pred = pipe.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, pred)

    # extract feature names + coefficients
    ohe = pipe.named_steps["pre"].named_transformers_["cat"]
    cat_names = ohe.get_feature_names_out(cat_cols).tolist() if len(cat_cols) else []
    feat_names = cat_names + num_cols
    coefs = pipe.named_steps["clf"].coef_.ravel()

    coef_df = pd.DataFrame({"feature": feat_names, "coef": coefs})
    coef_df["abs"] = coef_df["coef"].abs()
    coef_df = coef_df.sort_values("abs", ascending=False)

    print(f"Logistic model AUC: {auc:.3f}")
    return coef_df


def plot_top_coeffs(coef_df: pd.DataFrame, top_n=25):
    top = coef_df.head(top_n).sort_values("coef")
    fig = px.bar(top, x="coef", y="feature", orientation="h", title=f"Top {top_n} logistic coefficients (directional)")
    fig.update_layout(template="plotly_white", height=700, width=1100)
    fig.show()


# -------------------------
# 8) RUN EVERYTHING (ONE CALL)
# -------------------------
def run_full_eda(df: pd.DataFrame, time_basis="closed_month"):
    d = preprocess_sales_df(df)

    eda_overview(d)

    # Monthly & cumulative
    kpi = monthly_kpis(d, time_basis=time_basis)
    plot_monthly_kpis(kpi, title_prefix=f"Monthly KPIs using {time_basis}")

    # MoM changes
    z = mom_changes(kpi)
    plot_mom_changes(z)

    # Category drivers
    cat_cols = ["industry","region","product_type","lead_source","deal_stage","sales_rep_id"]
    for col in cat_cols:
        if col in d.columns:
            category_winrate_overall(d, col=col, top_n=12, min_n=30)
            category_winrate_monthly(d, col=col, time_basis=time_basis, top_n=8, min_n_per_month=10)
            category_winrate_heatmap(d, col=col, time_basis=time_basis, top_n=10, min_n_per_cell=8)

    # Continuous effects
    continuous_effects(d, time_basis=time_basis)

    # Cross-category heatmaps (pick a few common combos)
    combos = [("region","product_type"), ("lead_source","product_type"), ("industry","region"), ("deal_stage","lead_source")]
    for r, c in combos:
        if r in d.columns and c in d.columns:
            winrate_pivot(d, r, c, min_n=25)

    return d, kpi


In [10]:


# =========================
# USAGE
# =========================

d, kpi = run_full_eda(data, time_basis="closed_month")   # performance over time
d, kpi = run_full_eda(data, time_basis="created_month")  # pipeline creation over time

# Optional: driver model (directional)
coef_df = logistic_driver_model(d, time_basis="closed_month")
plot_top_coeffs(coef_df, top_n=30)


Shape: (5000, 16)

Missing %:
Series([], dtype: float64)

Outcome distribution:
0       2737
1       2263
<NA>       0
Name: outcome, dtype: Int64

Created date range: 2023-01-01 00:00:00 -> 2024-03-26 00:00:00
Closed  date range: 2023-01-11 00:00:00 -> 2024-07-20 00:00:00

Duplicate deal_id: 0


Shape: (5000, 16)

Missing %:
Series([], dtype: float64)

Outcome distribution:
0       2737
1       2263
<NA>       0
Name: outcome, dtype: Int64

Created date range: 2023-01-01 00:00:00 -> 2024-03-26 00:00:00
Closed  date range: 2023-01-11 00:00:00 -> 2024-07-20 00:00:00

Duplicate deal_id: 0


Logistic model AUC: 1.000


In [11]:
data["won_deal_amount"] = data["deal_amount"] * data["outcome"]

In [12]:
def monthly_weighted_win_rate(df, date_col="closed_date", outcome_col="outcome", amount_col="deal_amount"):
    d = df.copy()
    d[date_col] = pd.to_datetime(d[date_col], errors="coerce")
    d[outcome_col] = pd.to_numeric(d[outcome_col], errors="coerce")
    d[amount_col] = pd.to_numeric(d[amount_col], errors="coerce")

    d = d.dropna(subset=[date_col, outcome_col, amount_col])
    d["month"] = d[date_col].dt.to_period("M").dt.to_timestamp()

    g = d.groupby("month", as_index=False).apply(
        lambda x: pd.Series({
            "weighted_win_rate": (x[amount_col] * x[outcome_col]).sum() / x[amount_col].sum() if x[amount_col].sum() != 0 else np.nan,
            "count_win_rate": x[outcome_col].mean(),
            "closed_deals": len(x),
            "total_amount": x[amount_col].sum()
        })
    ).reset_index(drop=True)

    return g.sort_values("month")


monthly_weighted_win_rate_data = monthly_weighted_win_rate(data)

monthly_weighted_win_rate_data.head()

,month,weighted_win_rate,count_win_rate,closed_deals,total_amount
0,2023-01-01,0.208827,0.344828,29.0,453371.0
1,2023-02-01,0.397901,0.450549,91.0,2425603.0
2,2023-03-01,0.520751,0.456731,208.0,4810993.0
3,2023-04-01,0.404404,0.450355,282.0,7334158.0
4,2023-05-01,0.476490,0.468927,354.0,8580514.0


In [13]:
def plot_monthly_weighted_win_rate_plotly(
    mwr_df,
    title="Monthly Win Rate (Count vs Revenue-Weighted)"
):
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=mwr_df["month"],
            y=mwr_df["count_win_rate"],
            mode="lines+markers",
            name="Count win rate"
        )
    )

    fig.add_trace(
        go.Scatter(
            x=mwr_df["month"],
            y=mwr_df["weighted_win_rate"],
            mode="lines+markers",
            name="Revenue-weighted win rate"
        )
    )

    fig.update_layout(
        template="plotly_white",
        title=title,
        xaxis_title="Month",
        yaxis_title="Win rate",
        hovermode="x unified"
    )

    fig.update_yaxes(tickformat=".0%")
    fig.show()

# usage
plot_monthly_weighted_win_rate_plotly(monthly_weighted_win_rate_data)


In [14]:
dims = ["industry", "region", "product_type", "lead_source", "deal_stage"]
df = data.copy()

df["is_won"] = (df["outcome"] == 1).astype(int)

df["is_won"] = df["is_won"].astype(int)  # ensure 0/1

amount_col = "deal_amount"  # change if your column is different

g = (
    df.groupby(dims, dropna=False)
      .agg(
          total_deals=("is_won", "size"),
          wins=("is_won", "sum"),
          total_amount=(amount_col, "sum"),
          won_amount=(amount_col, lambda s: s[df.loc[s.index, "is_won"] == 1].sum()),
      )
      .reset_index()
)

g["win_rate"] = g["wins"] / g["total_deals"]

# Weighted win rate by amount
g["weighted_win_rate"] = np.where(
    g["total_amount"] > 0,
    g["won_amount"] / g["total_amount"],
    np.nan
)

# Optional: filter out tiny samples so "highest" isn't noise
MIN_DEALS = 7
g_f = g[g["total_deals"] >= MIN_DEALS].copy()

# Top combinations by win rate
top_win_rate = g_f.sort_values(["win_rate", "total_deals"], ascending=[False, False]).head(20)

# Top combinations by weighted win rate
top_weighted = g_f.sort_values(["weighted_win_rate", "total_amount"], ascending=[False, False]).head(20)



In [15]:
def _combo_label(df_, dims):
    # readable y-axis label for each combination
    return df_[dims].astype(str).agg(" | ".join, axis=1)

def plot_top_combos_dualbars(g_f, dims, top_n=20, sort_by="win_rate"):
    """
    Creates TWO horizontal bar charts (Plotly):
    1) Top combos sorted by win_rate
    2) Top combos sorted by weighted_win_rate

    Each chart plots BOTH win_rate and weighted_win_rate for the same combos.
    """
    # --- Chart 1: sorted by win_rate ---
    top1 = (g_f.sort_values(["win_rate", "total_deals"], ascending=[False, False])
              .head(top_n)
              .copy())
    top1["combo_label"] = _combo_label(top1, dims)
    top1 = top1.sort_values("win_rate", ascending=True)  # so best is at bottom in h-bar

    fig1 = go.Figure()
    fig1.add_trace(go.Bar(
        y=top1["combo_label"],
        x=top1["win_rate"],
        name="win_rate",
        orientation="h",
        hovertemplate=(
            "win_rate=%{x:.2%}<br>"
            "weighted_win_rate=%{customdata[0]:.2%}<br>"
            "wins=%{customdata[1]} / %{customdata[2]}<br>"
            "total_amount=%{customdata[3]:,.0f}<extra></extra>"
        ),
        customdata=list(zip(
            top1["weighted_win_rate"],
            top1["wins"],
            top1["total_deals"],
            top1["total_amount"]
        ))
    ))
    fig1.add_trace(go.Bar(
        y=top1["combo_label"],
        x=top1["weighted_win_rate"],
        name="weighted_win_rate",
        orientation="h",
        hovertemplate=(
            "weighted_win_rate=%{x:.2%}<br>"
            "win_rate=%{customdata[0]:.2%}<br>"
            "won_amount=%{customdata[1]:,.0f}<br>"
            "total_amount=%{customdata[2]:,.0f}<extra></extra>"
        ),
        customdata=list(zip(
            top1["win_rate"],
            top1["won_amount"],
            top1["total_amount"]
        ))
    ))

    fig1.update_layout(
        title=f"Top {top_n} combinations by Win Rate (showing both metrics)",
        barmode="group",
        xaxis=dict(title="Rate", tickformat=".0%"),
        yaxis=dict(title="industry | region | product_type | lead_source | deal_stage"),
        height=max(500, 24 * len(top1)),
        legend_title_text="Metric",
        margin=dict(l=260, r=30, t=60, b=40),
    )

    # --- Chart 2: sorted by weighted_win_rate ---
    top2 = (g_f.sort_values(["weighted_win_rate", "total_amount"], ascending=[False, False])
              .head(top_n)
              .copy())
    top2["combo_label"] = _combo_label(top2, dims)
    top2 = top2.sort_values("weighted_win_rate", ascending=True)

    fig2 = go.Figure()
    fig2.add_trace(go.Bar(
        y=top2["combo_label"],
        x=top2["win_rate"],
        name="win_rate",
        orientation="h",
        hovertemplate=(
            "win_rate=%{x:.2%}<br>"
            "weighted_win_rate=%{customdata[0]:.2%}<br>"
            "wins=%{customdata[1]} / %{customdata[2]}<br>"
            "total_amount=%{customdata[3]:,.0f}<extra></extra>"
        ),
        customdata=list(zip(
            top2["weighted_win_rate"],
            top2["wins"],
            top2["total_deals"],
            top2["total_amount"]
        ))
    ))
    fig2.add_trace(go.Bar(
        y=top2["combo_label"],
        x=top2["weighted_win_rate"],
        name="weighted_win_rate",
        orientation="h",
        hovertemplate=(
            "weighted_win_rate=%{x:.2%}<br>"
            "win_rate=%{customdata[0]:.2%}<br>"
            "won_amount=%{customdata[1]:,.0f}<br>"
            "total_amount=%{customdata[2]:,.0f}<extra></extra>"
        ),
        customdata=list(zip(
            top2["win_rate"],
            top2["won_amount"],
            top2["total_amount"]
        ))
    ))

    fig2.update_layout(
        title=f"Top {top_n} combinations by Weighted Win Rate (showing both metrics)",
        barmode="group",
        xaxis=dict(title="Rate", tickformat=".0%"),
        yaxis=dict(title="industry | region | product_type | lead_source | deal_stage"),
        height=max(500, 24 * len(top2)),
        legend_title_text="Metric",
        margin=dict(l=260, r=30, t=60, b=40),
    )

    return fig1, fig2

fig1, fig2 = plot_top_combos_dualbars(g_f, dims, top_n=20)
fig1.show()
fig2.show()
